본인의 DATA SET load는 여기서 알아서 진행하도록 하자.
==

Encoder/Decoder train / test set 작성.
---
#### 현재 소스에서는 Decoder에서 Y값을 이동시키며 교사강요 시키는 소스가 아니므로, Y data set도 있어야함

#### 예시 
enc_train.shape : (batch_size, time_step, feature_len) <br />
dec_train.shape : (batch_size, time_step, wanted input decoder data len)  <br />
y_train.shape : (batch_size, time_step, predict data len)  <br />
 <br />
enc_test.shape : (batch_size, time_step, feature_len)  <br />
dec_test.shape : (batch_size, time_step, wanted input decoder data len)  <br />
y_test.shape : (batch_size, time_step, predict data len)  <br />

총 6개의 변수가 필요하다.

In [ ]:
batch_size=1000
steps_per_epoch = len(enc_train)
buffer_size = len(enc_train)

train_ds = (
    tf.data.Dataset.from_tensor_slices(
        (enc_train, dec_train, y_train)
    )
    .batch(batch_size)
#     .shuffle(buffer_size=20000)
)

test_ds = tf.data.Dataset.from_tensor_slices(
    (enc_test, dec_test, y_test)
).batch(batch_size)

In [ ]:
%load_ext autoreload
%autoreload 2
from attention import *

# 0:내적어텐션 / 1:바다나우어텐션
attention_cate = 0
predict_len = 1
n_hidden = 128

'''
내 생각에 아직 구체적으로 테스트해보진 못했지만, loss 계산 부분이 뭔가 매 시간 loss를 계산한다던지 했으면,
좀 더 정확도 좋은 예측값을 얻어낼 수 있지 않을까도 싶다.(현재는 모든 time-step loss 계산후 한꺼번에 적용)
각 epoch당 loss가 효과적으로 변화되지 못하지 않을까? 하는 의문도 있다.
'''

if attention_cate == 0:
    encoder = attention_enc(n_hidden)
    decoder = dot_product_dec(n_hidden)
    
    enc_x, dec_x, dec_y = next(iter(train_ds))
    print(enc_x.shape, dec_x.shape, dec_y.shape)
    
    @tf.function
    def train_step(inputs,labels,loss_fn,optimizer):
        loss = 0
        encoder_input, decoder_input = inputs
        with tf.GradientTape() as tape:
            enc_output, enc_hidden, enc_c = encoder(encoder_input)
            
            # 교사강요를 위한 for문
            for t in range(0, decoder_input.shape[1]):
                # decoder의 time-step을 1개로 분리
                dec_input = tf.reshape(decoder_input[:,t],(-1,1,decoder_input.shape[2]))
                # 레퍼런스대로 enc값과 dec값을 이용하여 학습 시작
                predictions, attention_weights = decoder(dec_input, enc_output, enc_hidden, enc_c)
                # 교사강요때문에, 각 time-step에 대한 label을 predictions값과 바로바로 비교하여 로스를 더한다.
                loss += loss_fn(tf.reshape(labels[:,t],(-1,1,1)), predictions)
        batch_loss = tf.reduce_mean(loss)
        variables = encoder.trainable_variables + decoder.trainable_variables
        # 전부다 더해진 loss를 통하여 gradients를 조정하게된다.
        gradients = tape.gradient(loss,variables)
        optimizer.apply_gradients(zip(gradients,variables))

        return batch_loss

    loss_fn = keras.losses.mean_absolute_error
    optimizer = tf.keras.optimizers.Adam(0.001)
    
    import time
    
    for epoch in range(5000):
        start = time.time()
        total_loss = 0

        for (batch, (enc, dec, label)) in enumerate(train_ds.take(steps_per_epoch)):
            inputs = [enc,dec]
            batch_loss = train_step(inputs,label,loss_fn,optimizer)
            total_loss += batch_loss
            if batch % 100 == 0:
                print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,batch, batch_loss.numpy()))

        print('Epoch {} Loss {:.4f}'.format(epoch + 1, total_loss.numpy() / steps_per_epoch))
        print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))
        
elif attention_cate == 1:
    encoder = attention_enc(n_hidden)
    decoder = bahdanau_dec(n_hidden, 'v')
    
    enc_x, dec_x, dec_y = next(iter(train_ds))
    print(enc_x.shape, dec_x.shape, dec_y.shape)
    
    @tf.function
    def train_step(inputs,labels,loss_fn,optimizer):
        loss = None
        encoder_input, decoder_input = inputs
        with tf.GradientTape() as tape:
            enc_output, enc_hidden, enc_c = encoder(encoder_input)
            # 바다나우 어텐션은 t-1을 활용하는 성질때문에, 가장 처음은 enc_hidden을, 이후에는 dec_hidden을 사용해야해서
            # 전단에 이런 작업들이 발생한다.
            dec_hidden = enc_hidden
            dec_input = tf.expand_dims(decoder_input[:,0], 1)
            
            # range는 데이터 특성을 보고 0부터 시작할지, 1부터 시작할지 잘 정하도록 하자.
            for t in range(0, labels.shape[1]):
                predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_output)
                dec_hidden = dec_hidden[0]
                
                # loss로 개선할 수 있지 않을까 싶어서, loss를 단순 다 더하는 형태에서
                # 각 time-step 별로 loss를 배열화 해서 던져보기로 했다. 필요에 따라 바꿔서 사용하길 바란다.
#                 loss += loss_fn(labels[:,t], predictions)
                if loss == None:
                    loss = tf.expand_dims(loss_fn(labels[:,t], predictions),1)
                    print(loss.shape)
                else:
                    loss = tf.concat([loss,tf.expand_dims(loss_fn(labels[:,t], predictions),1)], axis=-1)
                print("after_transpose_shape : ", loss.shape)
#                 tf.print("loss : ",loss, summarize=-1, output_stream=sys.stdout)
                dec_input = tf.expand_dims(labels[:,t], 1)
        
        batch_loss = tf.reduce_mean(loss)
        variables = encoder.trainable_variables + decoder.trainable_variables
        # 배열로 쌓아서 time-step만큼의 loss를 던져도 계산은 잘된다.
        gradients = tape.gradient(loss,variables)
        optimizer.apply_gradients(zip(gradients,variables))

        return batch_loss

    loss_fn = keras.losses.MAE
    optimizer = tf.keras.optimizers.Adam(0.001)
    import time
    for epoch in range(10000):
        start = time.time()
        total_loss = 0

        for (batch, (enc, dec, label)) in enumerate(train_ds.take(steps_per_epoch)):
            inputs = [enc,dec]
            batch_loss = train_step(inputs,label,loss_fn,optimizer)
            total_loss += batch_loss
            if batch % 100 == 0:
                print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,batch, batch_loss.numpy()))

        print('Epoch {} Loss {:.4f}'.format(epoch + 1, total_loss.numpy() / steps_per_epoch))
        print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

In [ ]:
# 바다나우를 사용한 경우 해당 evaluate를 사용해 test 진행 가능하다.
def bada_evaluate (enc_x_test, dec_x_test, lookup):
    if lookup == 'h':
        plot_shape = (dec_x_test.shape[1], enc_x_test.shape[2])
    elif lookup == 'v':
        plot_shape = (dec_x_test.shape[1], enc_x_test.shape[1])
    attention_plot = np.zeros(plot_shape)
    enc_out, enc_hidden, _ = encoder(enc_x_test)
    dec_hidden = enc_hidden
    pred_result = []
    
    # 바다나우 특성상 0번째는 수동으로 한번 돌아가고,
    dec_input = tf.reshape(dec_x_test[:,0],(-1,1,dec_x_test.shape[2]))
    predictions, dec_hidden, attention_weights  = decoder(dec_input, dec_hidden, enc_out)
    pred_result.append(predictions)
    # weights도 0번째는 한번 저장한다.
    # attention_weights가 한 시점에 대한 lstm_output_feature만큼 나오게되니까, mean으로 간단하게 출력해보기로 했다.
    # 참고 : https://www.tensorflow.org/tutorials/text/nmt_with_attention?hl=ko
    attention_weights = tf.reduce_mean(attention_weights, axis=0)
    attention_plot[0] = attention_weights.numpy().squeeze()
    
    # decoder의 최종 time-step까지 돌면서 예측을 진행한다.
    # 이부분은 참고할 것이 학습을 decoder의 다음값을 교사강요 시키지 않고, 어떤 1:1매칭의 예측을 해야된다면, 다른 형태로 구현되어야 한다.
    for t in range(1, dec_x_test.shape[1]):
        # enc_output를 디코더에 전달합니다.
        print(predictions.numpy().shape)
        dec_hidden = dec_hidden[0]
        dec_input = tf.expand_dims(predictions, 1)
#         dec_input = tf.reshape(dec_x_test[:,t],(-1,1,dec_x_test.shape[2]))
#         dec_input = np.concatenate((dec_x_test[:,t,:-1].reshape(-1,1,dec_x_test[:,t,:-1].shape[1]),predictions.numpy()),axis=2)
        predictions, dec_hidden, attention_weights  = decoder(dec_input, dec_hidden, enc_out)
        attention_weights = tf.reduce_mean(attention_weights, axis=0)
        attention_plot[t] = attention_weights.numpy().squeeze()
        pred_result.append(predictions)
    return np.array(pred_result), attention_plot

In [ ]:
# 이런식으로 0번째 time-step에 대한 예측을 진행해볼 수도 있음.
enc_out, enc_hidden, _ = encoder(enc_test)
dec_hidden = enc_hidden
dec_input = tf.reshape(dec_test[:,0],(-1,1,dec_test.shape[2]))

predictions, dec_hidden, attention_weights  = decoder(dec_input, dec_hidden, enc_out)
print(predictions.numpy().shape)
dec_input = tf.expand_dims(predictions, 1)
dec_hidden = dec_hidden[0]
predictions, dec_hidden, attention_weights  = decoder(dec_input, dec_hidden, enc_out)

In [ ]:
# 예측값 출력
tf.print(predictions[0], output_stream=sys.stdout)

In [ ]:
# 내적을 사용한 경우 해당 evaluate를 사용해 test 진행 가능하다.
def evaluate (enc_x_test, dec_x_test, lookup):
    attention_plot = np.zeros(plot_shape)
    enc_out, enc_hidden, enc_c = encoder(enc_x_test)
    pred_result = []
    
    for t in range(0, dec_x_test.shape[1]):
        # decoder에서 feature라고 생각되는 값을 넣어서 다음값을 예측하게되는 형태로도 구현하려고 concat을 써서, 강제로 예측값을 제외한
        # 다른 특징값들을 합치는 작업들도 진행을 했었다.
        # 단순하게 값만 예측하는 형태라면 그냥 decoder의 t-1값을 가지고 t를 예측하도록, 학습자가 가르친 형태로 evaluate를 구현하면 된다.
        if t==0:
            dec_input = tf.reshape(dec_x_test[:,t],(-1,1,dec_x_test.shape[2]))
        else:
            dec_input = np.concatenate((dec_x_test[:,t,:-1].reshape(-1,1,dec_x_test[:,t,:-1].shape[1]),predictions.numpy()),axis=2)
        predictions, attention_weights  = decoder(dec_input, enc_out, enc_hidden, enc_c)
        attention_weights = tf.reduce_mean(attention_weights, axis=0)
        attention_plot[t] = attention_weights.numpy().squeeze()
        pred_result.append(predictions)
    return np.array(pred_result), attention_plot

In [ ]:
result, att_w = evaluate(enc_test,dec_test,'v')

In [ ]:
plt.rcParams["figure.figsize"] = (100,100)
plt.rcParams['axes.grid'] = True

# heatmap by plt.pcolor()
# xticks와 yticks는 본인들의 encoder decoder time-step을 고려하여 숫자를 집어넣도록 하자.
plt.pcolor(att_w)
plt.xticks(np.arange(0, 196, 1), list(range(0,196)), fontsize=100)
plt.yticks(np.arange(0, 98, 1), list(range(0,98)), fontsize=100)
plt.title('Heatmap by plt.pcolor()', fontsize=20)
plt.xlabel('Input_TimeSequence', fontsize=100)
plt.ylabel('Input_History_TimeSquence', fontsize=100)
plt.colorbar()

plt.show()
